In [20]:
import tkinter as tk
import sounddevice as sd
import numpy as np
import speech_recognition as sr
import scipy.io.wavfile as wav
import webbrowser

class SpeechToYouTubeApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Speech to YouTube Search")
        self.master.geometry("400x300")
        self.master.configure(bg="#ADD8E6")  # Light blue background color

        # Create a frame to center the button
        self.center_frame = tk.Frame(master, bg="#ADD8E6")
        self.center_frame.pack(expand=True)  # Center the frame in the window

        # Create a heart-shaped button
        self.record_button = tk.Button(self.center_frame, text="💖 Record Speech 💖", command=self.record_speech, bg="#FF1493", fg="white", font=("Arial", 16, "bold"), relief="flat", activebackground="#FFB6C1")
        self.record_button.pack(pady=20)
        self.record_button.bind("<Enter>", self.on_enter)
        self.record_button.bind("<Leave>", self.on_leave)

        # Create a label to display the recognized text
        self.result_label = tk.Label(master, text="", wraplength=300, bg="#ADD8E6", font=("Arial", 14))
        self.result_label.pack(pady=10)

        # Create a label to show the text "You searched for:"
        self.search_text_label = tk.Label(master, text="You searched for:", bg="#ADD8E6", font=("Arial", 14))
        self.search_text_label.pack(pady=10)

        # Create a label for the listening animation
        self.listening_label = tk.Label(master, text="", bg="#ADD8E6", font=("Arial", 14))
        self.listening_label.pack(pady=10)

        # Variable to keep track of recording status
        self.recording = False

    def on_enter(self, e):
        """Increase button size on hover."""
        self.record_button.config(font=("Arial", 20, "bold"))

    def on_leave(self, e):
        """Reset button size on leave."""
        self.record_button.config(font=("Arial", 16, "bold"))

    def record_audio(self, duration=5, sample_rate=44100):
        """Records audio from the microphone for the given duration (in seconds)."""
        print(f"Recording for {duration} seconds...")
        audio = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
        sd.wait()  # Wait until the recording is finished
        return audio, sample_rate

    def save_audio_to_wav(self, audio, sample_rate, filename="output.wav"):
        """Saves the recorded audio to a WAV file."""
        wav.write(filename, sample_rate, np.int16(audio * 32767))  # Convert to 16-bit PCM
        print(f"Audio saved to {filename}")

    def speech_to_text(self, filename="output.wav"):
        """Transcribes speech from an audio file using Google Speech Recognition."""
        recognizer = sr.Recognizer()
        
        with sr.AudioFile(filename) as source:
            audio_data = recognizer.record(source)  # Read the entire file

        try:
            text = recognizer.recognize_google(audio_data)
            print(f"You said: {text}")
            return text
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand the audio")
            return None
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
            return None

    def open_youtube_based_on_text(self, speech_text):
        """Redirects to YouTube based on the recognized speech text."""
        if not speech_text:
            print("No valid speech text to process.")
            return

        # Join the text to form a search query
        query = '+'.join(speech_text.split())
        base_url = "https://www.youtube.com/results?search_query="
        webbrowser.open(base_url + query)
        print(f"Opening YouTube with search: '{query}'")

    def animate_listening(self):
        """Animates the 'Listening' label."""
        if self.recording:
            self.listening_label.config(text="Listening", fg="blue")
            self.master.after(500, self.animate_listening)  # Change text after 500ms
        else:
            self.listening_label.config(text="")

    def record_speech(self):
        # Start recording
        self.recording = True
        self.animate_listening()

        # Step 1: Record and save audio
        audio, sample_rate = self.record_audio(duration=5)
        self.save_audio_to_wav(audio, sample_rate)

        # Step 2: Transcribe speech from the saved WAV file
        speech_text = self.speech_to_text()

        # Step 3: Update the label with the recognized text
        self.result_label.config(text=speech_text)

        # Step 4: Redirect based on the recognized text
        self.open_youtube_based_on_text(speech_text)

        # Stop listening animation
        self.recording = False

if __name__ == "__main__":
    root = tk.Tk()
    app = SpeechToYouTubeApp(root)
    root.mainloop()


Recording for 5 seconds...
Audio saved to output.wav
You said: Malayalam song
Opening YouTube with search: 'Malayalam+song'
Recording for 5 seconds...
Audio saved to output.wav
You said: Malayalam song
Opening YouTube with search: 'Malayalam+song'
Recording for 5 seconds...
Audio saved to output.wav
You said: Malayalam song illuminati YouTube
Opening YouTube with search: 'Malayalam+song+illuminati+YouTube'
